In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
from geopandas import GeoDataFrame
from sklearn import preprocessing

plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titlepad'] = 20
plt.rcParams['axes.labelsize'] = 12

#### No estoy seguro si esta bien, pero pruebo

In [2]:
propiedades = pd.read_csv('properati.csv')

In [3]:
def obtener_barrio(fila):
    barrio,padre = fila
    return padre.split('|')[3]

In [4]:
propiedades.loc[:, 'place_name'] = propiedades.loc[:, ['place_name','place_with_parent_names']].apply(lambda x: obtener_barrio(x), axis = 1)

In [5]:
tipo_propiedad = {'PH': [0], 'apartment':[1], 'house': [2], 'store': [3]}

In [6]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipo_propiedad[x]) 

In [7]:
barrios = {}
x = 0
for i in propiedades.groupby('place_name').agg(np.mean).lat.keys():
    if i in barrios:
        continue
    barrios[i] = [x]
    x+=1

In [8]:
propiedades.reset_index(drop = True, inplace = True)

In [12]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: barrios[x]) 

In [14]:
from sklearn import preprocessing
tipos = preprocessing.OneHotEncoder()
tipos.fit(tipo_propiedad.values()) 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [11]:
preparacion = barrios.values()
for i in range (0, len(barrios)):
    preparacion[i] = preparacion[i]

In [15]:
pre_barrios = preprocessing.OneHotEncoder()
pre_barrios.fit(barrios.values())

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [16]:
propiedades.loc[:,'place_name'] = propiedades.loc[:,'place_name']\
.apply(lambda x: pre_barrios.transform([x]))

In [17]:
propiedades.loc[:,'property_type'] = propiedades.loc[:,'property_type']\
.apply(lambda x: tipos.transform([x]))

In [18]:
adivinar = propiedades.loc[propiedades.mes >= 7,:]

In [19]:
entrenar = propiedades.loc[propiedades.mes < 7,:]

In [20]:
condiciones = []
precios = []

for i in range(0,len(entrenar)):
    if np.isnan(entrenar.price_aprox_usd[i]):
        continue
    if np.isinf(entrenar.ditancia_subtes[i]):
        continue
    l = list(entrenar.place_name[i].toarray()[0]) + list(entrenar.property_type[i].toarray()[0])
    l.append(entrenar.ditancia_subtes[i])
    l.append(entrenar.seguridad[i])
    l.append(entrenar.gimnasio[i])
    condiciones.append(l)
    precios.append(entrenar.price_aprox_usd[i])

In [36]:
def adivinar_precio_indice(fila, estimador):
    barrio,tipo,subte, seg, gim = fila
    l =  list(barrio.toarray()[0]) + list(tipo.toarray()[0])
    if np.isinf(subte):
        return 0 # Ver de resolver estos despues
    l.append(subte)
    l.append(seg)
    l.append(gim)
    return  estimador.predict([l])[0]

In [59]:
def obtener_rmse(col_true, col_pred):
    y_true = []
    y_pred = []
    for i in range (0,len(adivinar)):
        if np.isnan(adivinar[col_true][i]):
            continue
        y_true.append(adivinar[col_true][i])
        y_pred.append(adivinar[col_pred][i])
    return mean_squared_error(y_true, y_pred)**0.5

In [25]:
from sklearn.neural_network import MLPRegressor

In [27]:
clf = MLPRegressor()
clf.fit(condiciones, precios)

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [39]:
adivinar.loc[:,'precio_adivinado_perceptron'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [43]:
from sklearn.metrics import mean_squared_error
#RMSE = mean_squared_error(adivinar.price_aprox_usd, adivinar.precio_adivinado_perceptron)**0.5

In [48]:
adivinar.reset_index(drop = True, inplace = True)

In [63]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron')

416926.48116043647

## Con otras condiciones

In [50]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(condiciones, precios)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [51]:
adivinar.loc[:,'precio_adivinado_perceptron2'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [62]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron2')

405050.72843388253

## Con otras condiciones

In [56]:
clf = MLPRegressor(tol=1e-6)
clf.fit(condiciones, precios)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [57]:
adivinar.loc[:,'precio_adivinado_perceptron3'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [61]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron3')

416971.59376647207

## Con otras condiciones

In [64]:
clf = MLPRegressor(solver='lbfgs',tol=1e-6)
clf.fit(condiciones, precios)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [65]:
adivinar.loc[:,'precio_adivinado_perceptron4'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [66]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron4')

395604.22924866236

## Con otras condiciones

In [67]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,),tol=1e-6)
clf.fit(condiciones, precios)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [68]:
adivinar.loc[:,'precio_adivinado_perceptron5'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [69]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron5')

395576.6010894691

## Con otras condiciones

In [147]:
clf = MLPRegressor(solver= 'lbfgs',activation = 'relu',tol=1e-10, max_iter = 500)
clf.fit(condiciones, precios)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=1e-10, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [149]:
adivinar.loc[:,'precio_adivinado_perceptron6'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [150]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron6')

394989.94872885587

In [ ]:
395397.13710463431 <- con relu y lbfgs   1e-10

395355.36639023374

## Con otras condiciones

In [151]:
clf = MLPRegressor(solver= 'lbfgs',activation = 'relu',tol=1e-10, max_iter = 500, warm_start = True)

In [152]:
for i in range (10):
    clf.fit(condiciones, precios)

In [153]:
adivinar.loc[:,'precio_adivinado_perceptron7'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice(x, clf), axis = 1)

In [154]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron7')

394903.38485623029

## Con preprocesamiento de standarizacion

In [155]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(condiciones)  
X_train = scaler.transform(condiciones)  

In [201]:
clf = MLPRegressor(solver= 'lbfgs',activation = 'relu',tol=1e-10, max_iter = 500,  warm_start = True)
#clf.fit(X_train, precios)

In [202]:
for i in range (20):
    clf.fit(X_train, precios)

In [166]:
def adivinar_precio_indice_stand(fila, estimador):
    barrio,tipo,subte, seg, gim = fila
    l =  list(barrio.toarray()[0]) + list(tipo.toarray()[0])
    if np.isinf(subte):
        return 0 # Ver de resolver estos despues
    l.append(subte)
    l.append(seg)
    l.append(gim)
    return  estimador.predict(scaler.transform([l]))[0]

In [203]:
adivinar.loc[:,'precio_adivinado_perceptron8'] = adivinar.loc[:, ['place_name','property_type','ditancia_subtes', 'seguridad', 'gimnasio']]\
.apply(lambda x: adivinar_precio_indice_stand(x, clf), axis = 1)

In [204]:
obtener_rmse('price_aprox_usd', 'precio_adivinado_perceptron8')

395422.71348460193